In [ ]:
# Interactive Figure
# %matplotlib ipympl 
# Non interactive
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from qualang_tools.units import unit
u = unit(coerce_to_integer=True)
from qm.qua import *
from qm import QuantumMachinesManager
from qm import SimulationConfig
from qualang_tools.results import fetching_tool
from qualang_tools.loops import from_array
import config_00 as config
from progress import addjob

In [ ]:
# Connect to the cluster (run only once)
import QM_cluster
qmm = QuantumMachinesManager(host=QM_cluster.QM_Router_IP, cluster_name=QM_cluster.cluster_name)

# Connexion à la Quantum Machine

In [ ]:
# Get the QM reference (rerun every time the config is changed)
qm_list =  qmm.list_open_qms()
qm = qmm.get_qm(qm_list[0])
print(f"Connected to {qm.id}")

# Mesure de la transmission dans le plan $IQ$ 

In [ ]:
n_points = 2048
with program() as prog:
    i = declare(int)
    n = declare(int)
    I1 = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q1 = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    I1_st = declare_stream()  # Stream for the 'I' quadrature
    Q1_st = declare_stream()  # Stream for the 'Q' quadrature
    I2 = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q2 = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    I2_st = declare_stream()  # Stream for the 'I' quadrature
    Q2_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_points, n + 1):
        # Play a weak pulse
        reset_frame("qubit")
        reset_frame("scope")
        play("pulse"*amp(0.05), "qubit")
        # Demodulate the signals to get the 'I' & 'Q' quadratures)
        measure(
            "short_readout",
            "scope",
            dual_demod.full("cos", "sin", I1),
            dual_demod.full("minus_sin", "cos", Q1),
        )
        align()
        # Play a second puilse rotated in the IQ plane
        frame_rotation_2pi(0.05, "qubit")
        play("pulse"*amp(0.05), "qubit")
        measure(
            "short_readout",
            "scope",
            dual_demod.full("cos", "sin", I2),
            dual_demod.full("minus_sin", "cos", Q2),
        )        
        # Save the 'I' & 'Q' quadratures to their respective streams
        save(I1, I1_st)
        save(Q1, Q1_st)
        save(I2, I2_st)
        save(Q2, Q2_st)

    with stream_processing():
        # Cast the data into a 1D vector, and store the results on the OPX processor
        I1_st.buffer(n_points).save("I1")
        Q1_st.buffer(n_points).save("Q1")
        I2_st.buffer(n_points).save("I2")
        Q2_st.buffer(n_points).save("Q2")


In [ ]:
# Run the code and fetch results
job = addjob(prog, qm)
res = fetching_tool(job, ['I1','Q1','I2','Q2'])
I1, Q1, I2, Q2 = res.fetch_all()
print('Job done')

In [ ]:
# Plot
fig,ax=plt.subplots()
ax.plot(I1,Q1,',')
ax.plot(I2,Q2,',')
ax.set_aspect('equal')

# Exercices

## Exercice 1: Histogrammes 
- Tourner les données dans le plan IQ pour que la différence entre les deux jeux de données soit codée sur une seule quadrature
- Tracer les histogrammes de la quadrature ainsi obtenue
 
## Exercise 2: Erreur de lecture
- Estimer la probabilité d'assigner un point expérimental au mauvais jeu de données
- Trouver expérimentalement l'angle de rotation minimal tel que l'erreur de lecture soit inférieure à 5%